In [5]:
import pandas as pd
import datetime
import numpy as np

#fields = ['timestamp', 'replies','likes', 'retweets' ,'text']
# clean_tweets_part_2.csv #Contains data from 2015-11-03 to 2017-02-17
filename = 'gs://bigdata-general/clean/clean_tweets_part_0.csv'

tweets = pd.read_csv(filename, delimiter=',', index_col=0, nrows=30000)
# Convert timestamp to date object
tweets['timestamp'] = pd.to_datetime(tweets['timestamp']).dt.date
tweets

,timestamp,replies,likes,retweets,text
0,2019-05-27,0,0,0,È appena uscito un nuovo video! LES CRYPTOMONN...
1,2019-05-27,0,0,0,Cardano: Digitize Currencies; EOS https://t.co...
2,2019-05-27,0,2,1,Another Test tweet that wasn't caught in the s...
3,2019-05-27,0,0,0,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...
4,2019-05-27,0,0,0,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...
...,...,...,...,...,...
29995,2019-05-10,0,0,0,Buy £200 worth of bitcoin! Enjoy #DearMeTenYe...
29996,2019-05-10,0,0,0,Channel: #VidsCrypto\nVideo Title: youtubecorr...
29997,2019-05-10,0,0,0,@WormHarkin Too long my friend. Recently liqui...
29998,2019-05-10,0,0,0,I don't even own #bitcoin but it makes me happ...


# Cleaning Text from tweets

In [6]:
import re
import emoji
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())

def cleaner(tweet):
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet) #Remove non-ASCII characters
    tweet = re.sub(r'[^\w]', ' ', tweet) #Remove Symbols and punctuation
    tweet = " ".join(tweet.split())
    #tweet = ''.join(c for c in tweet if c not in emoji.UNICODE_EMOJI) #Remove Emojis
    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) \
         if w.lower() in words or not w.isalpha())
    return tweet


[nltk_data] Downloading package words to /home/amaru-
[nltk_data]     razerblade/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [7]:
tweets['text'] = tweets['text'].map(lambda x: cleaner(x))

In [8]:
# # Removing empty tweets
tweets = tweets[~tweets.text.eq('')]


In [9]:
tweets.reset_index(drop=True)

,timestamp,replies,likes,retweets,text
0,2019-05-27,0,0,0,un video EN 2019
1,2019-05-27,0,0,0,Digitize 6500 ROI AT T Bill Pay follo4folloback
2,2019-05-27,0,2,1,Another Test tweet that t caught in the stream
3,2019-05-27,0,0,0,Current 8721 99 266 62 114 03 432 02 0 133 DOG...
4,2019-05-27,0,0,0,Spiv Is An Asset NOT A Currency
...,...,...,...,...,...
27513,2019-05-10,0,0,0,Buy 200 worth of Enjoy
27514,2019-05-10,0,0,0,Channel Video Title
27515,2019-05-10,0,0,0,Too long my friend Recently immediately before...
27516,2019-05-10,0,0,0,I don t even own but it me happy when it goes ...


# Labeling Cleaned tweets

In [10]:
# Sort dataframe set by timestamp and reindex
#
tweets = tweets.sort_values(by=['timestamp'])
tweets = tweets.reset_index(drop=True)

In [11]:
# Load historical bitcoin data from the following file located in gs://bigdata-general/bitcoin_historical_price.csv
# Parse date column as a date

bitcoin = pd.read_csv("gs://bigdata-general/bitcoin_historical_price.csv", index_col=0, parse_dates=True)

In [12]:
# Remove time from Date
bitcoin['Date'] = pd.to_datetime(bitcoin['Date']).dt.date

In [13]:
# calculate daily percentage change from opening to close.
daily_change = (bitcoin['Close'] - bitcoin['Open'])/bitcoin['Open']*100
# Label daily_change depending on the following conditions:
#     if daily_change is less than -10, label it as 0
#     if daily_change is in range -10 to -3, label it as 1
#     if daily_change is in range -3 to 3, label it as 2
#     if daily_change is in range 3 to 10, label it as 3
#     if daily_change is greater than 10, label it as 4

# create a new column in the dataframe
bitcoin['%Daily Chg'] = daily_change
bitcoin['label'] = np.where(daily_change < -10, 0,
                   np.where(daily_change < -3, 1,
                   np.where(daily_change < 3, 2,
                   np.where(daily_change < 10, 3, 4))))

bitcoin

                    

,Name,Symbol,Date,High,Low,Open,Close,Volume,Marketcap,%Daily Chg,label
SNo,,,,,,,,,,,
1,Bitcoin,BTC,2013-04-29,147.488007,134.000000,134.444000,144.539993,0.000000e+00,1.603769e+09,7.509441,3
2,Bitcoin,BTC,2013-04-30,146.929993,134.050003,144.000000,139.000000,0.000000e+00,1.542813e+09,-3.472222,1
3,Bitcoin,BTC,2013-05-01,139.889999,107.720001,139.000000,116.989998,0.000000e+00,1.298955e+09,-15.834534,0
4,Bitcoin,BTC,2013-05-02,125.599998,92.281898,116.379997,105.209999,0.000000e+00,1.168517e+09,-9.597868,1
5,Bitcoin,BTC,2013-05-03,108.127998,79.099998,106.250000,97.750000,0.000000e+00,1.085995e+09,-8.000000,1
...,...,...,...,...,...,...,...,...,...,...,...
2987,Bitcoin,BTC,2021-07-02,33939.588699,32770.680780,33549.600177,33897.048590,3.872897e+10,6.354508e+11,1.035626,2
2988,Bitcoin,BTC,2021-07-03,34909.259899,33402.696536,33854.421362,34668.548402,2.438396e+10,6.499397e+11,2.404788,2
2989,Bitcoin,BTC,2021-07-04,35937.567147,34396.477458,34665.564866,35287.779766,2.492431e+10,6.615748e+11,1.794908,2


In [14]:
labeled = []
labeled = pd.DataFrame(labeled)
labeled['Date'] = bitcoin['Date']
labeled['Daily %Chg'] = daily_change
labeled['Cat.'] = bitcoin['label']
labeled = labeled.reset_index(drop=True)
labeled

,Date,Daily %Chg,Cat.
0,2013-04-29,7.509441,3
1,2013-04-30,-3.472222,1
2,2013-05-01,-15.834534,0
3,2013-05-02,-9.597868,1
4,2013-05-03,-8.000000,1
...,...,...,...
2986,2021-07-02,1.035626,2
2987,2021-07-03,2.404788,2
2988,2021-07-04,1.794908,2
2989,2021-07-05,-4.359843,1


## Create a dictionary of historical bitcoin price changes by dat from lists

In [15]:
from dateutil import parser

dates = labeled.Date.values
categories = labeled['Cat.'].values

# Create a dictionary with dates as keys and categories as values
date_cat_dict = dict(zip(dates, categories))

# Create a list of dates
date = parser.parse('2021-06-28').date()
print(date_cat_dict[date])

2


## Add Category label to each Tweet

In [16]:
tweets['Date'] = tweets.timestamp.rename('Date')
# Drop column timestamp from tweets
tweets = tweets.drop(columns=['timestamp'])

In [ ]:
# Set value of column 'Category' if 'Date' matches the given date
# #
# date = parser.parse('2011-01-09').date()
# # Change 'Category' column values to 3 if 'Date' matches the given date
# tweets[tweets['Date'] == date]['Category'].values[0] = 3

# # Replace values based on conditions 
# # https://kanoki.org/2019/07/17/pandas-how-to-replace-values-based-on-conditions/
# tweets.loc[(tweets.Date == date), 'Category'] = 2

# # Print the number of tweets labeled as '3'
# print(tweets['Category'].value_counts())


In [17]:
# Match the timestamp of the tweets with bitcoin price and add a category label to the tweets
for key, value in date_cat_dict.items():
    tweets.loc[(tweets['Date'] == key), 'Category'] = value

In [18]:
 # Drop rows with missing Category value
 tweets = tweets[tweets['Category'].notnull()]
 tweets = tweets.reset_index(drop=True)
 tweets

,replies,likes,retweets,text,Date,Category
0,19,267,155,Central Banking Debt Serfdom This is as simple...,2017-01-27,2.0
1,3,5,3,How China Probably what today,2017-09-08,1.0
2,19,1248,434,I would not be interested in if t want to ban it,2017-09-14,0.0
3,12,509,238,1 While there are no a about the path a moneta...,2017-11-28,2.0
4,5,135,110,How many days it took to get from 0 1000 1789 ...,2017-11-29,2.0
...,...,...,...,...,...,...
27512,2,1,1,With so much good news coming is about to moon...,2019-05-27,2.0
27513,1,5,3,si na od 12 a 9000 price w jest od,2019-05-27,2.0
27514,1,27,4,5 17,2019-05-27,2.0
27515,0,0,0,Whale 4 000 000 worth of bought at 8 854 22 12...,2019-05-27,2.0


In [19]:
tweets[tweets.Category == 0]

,replies,likes,retweets,text,Date,Category
2,19,1248,434,I would not be interested in if t want to ban it,2017-09-14,0.0


In [20]:
#clean_tweets_part_0.csv
tweets.to_csv('/home/amaru-razerblade/clean_text_tweets/clean_text_tweets_part_0.csv')

In [21]:
!cd '/home/amaru-razerblade/clean_text_tweets' && gsutil cp clean_text_tweets_part_0.csv gs://bigdata-general/clean_text/clean_text_tweets_part_0.csv

Copying file://clean_text_tweets_part_0.csv [Content-Type=text/csv]...

Operation completed over 1 objects/2.5 MiB.                                      


# Util functions

In [ ]:
time = tweets['timestamp'][0]
# convert string to date object
time = datetime.datetime.strptime(time, '%Y-%m-%d').date()
# conver time to date
labeled['Date'].values[0]
time

In [102]:
d = '2013-04-29'
# convert d to a datetime object
d = datetime.datetime.strptime(d, '%Y-%m-%d').date()

val = labeled.loc[labeled.Date == r_date]['Cat.'].values[0]
val

2